In [1]:
from pathlib import Path
import os
import copy

In [2]:
fp = os.path.join(Path().absolute(), "inputs", "input23.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input23_test.txt")

with open(fp, "r") as f:
    data = f.read().split("\n")[:-1]

In [3]:
data

['#.###########################################################################################################################################',
 '#.#...#...###.....#...###...#...#.......#.....#.............#.........###...............#.......#.....#.....#.....#.......#...#.....###.....#',
 '#.#.#.#.#.###.###.#.#.###.#.#.#.#.#####.#.###.#.###########.#.#######.###.#############.#.#####.#.###.#.###.#.###.#.#####.#.#.#.###.###.###.#',
 '#...#.#.#.....#...#.#.....#.#.#.#.....#.#...#.#.......#.....#.#.......#...#.............#.....#.#...#...#...#.#...#.....#.#.#.#...#...#.#...#',
 '#####.#.#######v###.#######.#.#.#####.#.###.#.#######.#.#####.#.#######.###.#################.#.###.#####.###.#.#######.#.#.#.###.###.#.#.###',
 '#.....#...#...#.>.#.......#.#.#.#####.#...#.#.#...#...#.#.>.>.#...#...#.#...#.........#.......#.###...#...###.#.#.......#.#.#...#...#.#.#.###',
 '#.#######.#.#.#v#.#######.#.#.#.#####.###.#.#.#.#.#.###.#.#v#####.#.#.#.#.###.#######.#.#######.#####.#.#####.#.#.#######.

# Part 1

In [4]:
num_rows = len(data)
num_cols = len(data[0])

In [5]:
SLOPES = ["<", ">", "^", "v"]
ALLOWED = ["."] + SLOPES

def get_neighbours(point, with_slopes=True):
    x, y = point

    if with_slopes and data[x][y] in SLOPES:
        if data[x][y] == "^" and x > 0:
            neighbours = [(x - 1, y)]
        elif data[x][y] == "v" and x < num_rows - 1:
            neighbours = [(x + 1, y)]
        elif data[x][y] == "<" and y > 0:
            neighbours = [(x, y - 1)]
        elif data[x][y] == ">" and y < num_cols - 1:
            neighbours = [(x, y + 1)]

    else:
        neighbours = []
        if x > 0:
            cand = (x - 1, y)
            if data[cand[0]][cand[1]] in ALLOWED:
                neighbours.append(cand)
        if x < num_rows - 1:
            cand = (x + 1, y)
            if data[cand[0]][cand[1]] in ALLOWED:
                neighbours.append(cand)
        if y > 0:
            cand = (x, y - 1)
            if data[cand[0]][cand[1]] in ALLOWED:
                neighbours.append(cand)
        if y < num_cols - 1:
            cand = (x, y + 1)
            if data[cand[0]][cand[1]] in ALLOWED:
                neighbours.append(cand)

    return neighbours

In [6]:
start_x = 0
start_y = data[0].index(".")
print(start_x, start_y)
start = (start_x, start_y)

end_x = num_rows - 1
end_y = data[num_rows - 1].index(".")
print(end_x, end_y)
end = (end_x, end_y)

0 1
140 139


In [7]:
def build_neighbours_dict(with_slopes):
    neighbours_dict = {}
    for x in range(num_rows):
        for y in range(num_cols):
            point = (x, y)
            neighbours = get_neighbours(point, with_slopes=with_slopes)
            neighbours_dict[point] = neighbours
    
    return neighbours_dict

In [8]:
with_slopes = True

In [9]:
neighbours_dict = build_neighbours_dict(with_slopes)

In [10]:
# TOO SLOW for part 2

# exhaustive depth-first search
current = start
current_path = [start]
forks = []


# keep track of all forks encountered on current path along with children explored so far
# e.g. [(p1, {p2: visited, p3: unvisited}, index_of_parent_in_path), ...]

max_num_iter = 10000000
num_iter = 0
longest_path_length = -float("inf")

num_paths_explored = 0

while num_iter < max_num_iter:
    # print(f"{num_iter = }")
    # print(current_path)

    if num_iter % 1000 == 0:
        print(num_iter, num_paths_explored)

    if current == end:
        at_end = True
        if len(current_path) > longest_path_length:
            longest_path_length = len(current_path)
            print(f"new {longest_path_length = }")
    else:
        at_end = False

    neighbours = neighbours_dict[current]
    neighbours_excl_repetitions = [n for n in neighbours if n not in current_path]
    # if len([n for n in neighbours if n in current_path[:-2]]) > 0:
    #     print("cyclic", current_path)
    if len(neighbours_excl_repetitions) == 0 or at_end:

        # if len(neighbours_excl_repetitions) == 0:
        #     print("No more neighbours", current)

        num_paths_explored += 1

        # backtrack
        last_fork_index_old = last_fork_index
        
        search_terminated = True
        for fork_index in range(last_fork_index, -1, -1):
            unvisited_children = [ch for ch, visited_status in forks[fork_index][1].items() if visited_status is False]
            if len(unvisited_children) > 0:
                current = unvisited_children[0]
                last_fork_index = fork_index
                current_path = current_path[:(forks[last_fork_index][2] + 1)] + [current]
                forks = forks[:(last_fork_index + 1)]
                forks[last_fork_index][1][current] = True
                search_terminated = False
                break
        
        # if last_fork_index <= 10:
        # print(f"backtracking to last_fork_index {last_fork_index} / {last_fork_index_old}")
        if search_terminated:
            break

    elif len(neighbours_excl_repetitions) == 1:
        current = neighbours_excl_repetitions[0]
        current_path.append(current)

    elif len(neighbours_excl_repetitions) > 1:
        new_fork = (current, {n: False for n in neighbours_excl_repetitions}, len(current_path) - 1)

        current = neighbours_excl_repetitions[0]
        new_fork[1][current] = True
        forks.append(new_fork)
        last_fork_index = len(forks) - 1
        
        current_path.append(current)

        # print(f"Adding new fork: {forks = }")


    num_iter += 1

0 0
1000 0
new longest_path_length = 1931
2000 6
3000 14
4000 21
5000 29
6000 35
7000 44
8000 51
9000 58
10000 65
11000 74
12000 78
13000 87
14000 94
15000 98
16000 104
17000 114
18000 121
19000 129
20000 136
21000 144
new longest_path_length = 2011
22000 149
23000 158
24000 165
25000 169
26000 176
27000 185
28000 193
29000 200
30000 206
31000 214
32000 218
33000 228
34000 234
35000 240
36000 246
37000 253
38000 261
39000 265
40000 274
41000 281
42000 289
43000 296
44000 304
45000 309
46000 318
47000 325
48000 329
49000 338
50000 346
51000 354
52000 362
53000 367
54000 375
55000 380
56000 389
57000 397
58000 401
59000 409
60000 414
61000 423
62000 427
63000 435
64000 443
65000 450
66000 456
67000 464
68000 469
69000 479
70000 485
71000 491
72000 499
73000 504
74000 513
75000 518
76000 526
77000 532
78000 540
79000 545
80000 553
81000 558
82000 567
83000 572
84000 577
85000 583
86000 591
87000 599
88000 605
89000 613
90000 621
91000 628
92000 636
93000 641
94000 649
95000 655
96000 664


In [11]:
longest_path_length - 1

2010

# Part 2

Extract all forks to make the subsequent depth-first search faster (each node will be a fork), i.e. do edge contraction. Otherwise DFS will be too slow if it operates on all cells (even if they're not forks).

In [12]:
with_slopes = False

In [13]:
def build_adj_dict(with_slopes):

    adj_dict = {}

    for x in range(num_rows):
        for y in range(num_cols):
            point = (x, y)
            if data[x][y] != "#" or point in [start, end]:
                neighbours = get_neighbours(point, with_slopes=with_slopes)
                if len(neighbours) > 2 or point in [start, end]:
                    # this is a fork (or start or end)
                    # follow to next fork
                    start_fork = point
                    adj_dict[start_fork] = {}
                    for neighbour in neighbours:
                        current = neighbour
                        current_path = [start_fork, current]

                        while True:
                            neighbours = get_neighbours(current, with_slopes=with_slopes)
                            neighbours_excl_prev = [n for n in neighbours if n != current_path[-2]]
                            if len(neighbours_excl_prev) == 1:
                                current = neighbours_excl_prev[0]
                                current_path.append(current)
                            elif len(neighbours_excl_prev) >= 2:
                                # encountered fork
                                end_point = current
                                # current_path.append(current)
                                break
                            elif current in [start, end]:
                                end_point = current
                                # current_path.append(current)
                                break
                            else:
                                raise ValueError("cul-de-sac found")

                        adj_dict[start_fork][end_point] = len(current_path) - 1

    return adj_dict

In [14]:
adj_dict = build_adj_dict(with_slopes)

In [15]:
num_forks = 0
for x in range(num_rows):
    for y in range(num_cols):
        if data[x][y] != "#":
            n = get_neighbours((x, y), with_slopes=False)
            if len(n) > 2:
                # print(len(n))
                num_forks += 1
                # print((x, y), n)

print(num_forks)

34


In [16]:
for fork, neighbouring_forks in adj_dict.items():
    for neighbouring_fork, dist in neighbouring_forks.items():
        assert fork in adj_dict[neighbouring_fork] and adj_dict[neighbouring_fork][fork] == dist

This solution is a bit slow.

In [17]:
def do_dfs_over_forks(adj_dict, start, end):

    # exhaustive depth-first search
    current = start
    current_path = [start]
    forks = []

    # keep track of all forks encountered on current path along with children explored so far
    # e.g. [(p1, {p2: visited, p3: unvisited}, index_of_parent_in_path, path_length_so_far), ...]

    max_num_iter = 1000000000
    num_iter = 0
    longest_path_length = -float("inf")
    current_path_length = 0

    longest_path = None

    num_paths_explored = 0

    while num_iter < max_num_iter:
        # print(f"{num_iter = }")
        # print(current_path)

        if num_iter % 100000 == 0:
            print(num_iter, num_paths_explored)

        if current == end:
            at_end = True
            if current_path_length > longest_path_length:
                longest_path_length = current_path_length
                print(f"new {longest_path_length = }")
                longest_path = copy.deepcopy(current_path)
        else:
            at_end = False

        neighbours = adj_dict[current]
        neighbours_excl_repetitions = [n for n in neighbours if n not in current_path]
        if len(neighbours_excl_repetitions) == 0 or at_end:

            num_paths_explored += 1

            # backtrack
            last_fork_index_old = last_fork_index
            
            search_terminated = True
            for fork_index in range(last_fork_index, -1, -1):
                prev_fork = forks[fork_index]
                prev_fork_parent = prev_fork[0]
                unvisited_children = [ch for ch, visited_status in prev_fork[1].items() if visited_status is False]
                if len(unvisited_children) > 0:
                    current = unvisited_children[0]
                    last_fork_index = fork_index

                    dist_last_fork_to_child = adj_dict[prev_fork_parent][current]
                    current_path_length = forks[last_fork_index][3] + dist_last_fork_to_child

                    current_path = current_path[:(forks[last_fork_index][2] + 1)] + [current]
                    forks = forks[:(last_fork_index + 1)]
                    forks[last_fork_index][1][current] = True
                    search_terminated = False
                    break
            
            if last_fork_index <= 5:
                print(f"backtracking to last_fork_index {last_fork_index} / {last_fork_index_old}")
            if search_terminated:
                break

        elif len(neighbours_excl_repetitions) == 1:
            next_neighbour = neighbours_excl_repetitions[0]
            dist = adj_dict[current][next_neighbour]
            current = next_neighbour
            current_path.append(current)
            current_path_length += dist

        elif len(neighbours_excl_repetitions) > 1:
            new_fork = (current, {n: False for n in neighbours_excl_repetitions}, len(current_path) - 1, current_path_length)
            next_neighbour = neighbours_excl_repetitions[0]
            dist = adj_dict[current][next_neighbour]

            current = next_neighbour
            new_fork[1][current] = True
            forks.append(new_fork)
            last_fork_index = len(forks) - 1
            
            current_path.append(current)
            current_path_length += dist

            # print(f"Adding new fork: {forks = }")

        num_iter += 1

    return longest_path, longest_path_length

In [18]:
longest_path, longest_path_length = do_dfs_over_forks(adj_dict, start, end)

0 0
new longest_path_length = 4826
new longest_path_length = 4994
new longest_path_length = 5018
new longest_path_length = 5074
new longest_path_length = 5194
new longest_path_length = 5198
new longest_path_length = 5214
new longest_path_length = 5382
new longest_path_length = 5406
new longest_path_length = 5462
new longest_path_length = 5626
new longest_path_length = 5650
new longest_path_length = 5706
new longest_path_length = 5734
new longest_path_length = 5782
new longest_path_length = 5802
new longest_path_length = 5850
new longest_path_length = 5906
new longest_path_length = 5926
new longest_path_length = 5974
new longest_path_length = 6006
100000 29062
new longest_path_length = 6042
200000 58074
300000 85358
400000 113498
500000 143545
600000 174740
700000 202253
800000 230417
backtracking to last_fork_index 5 / 11
900000 259236
1000000 288737
1100000 318767
1200000 348918
1300000 378752
1400000 406450
1500000 432663
1600000 459865
backtracking to last_fork_index 4 / 9
1700000 4

In [19]:
assert len(longest_path) == len(set(longest_path))

In [20]:
longest_path[-10:]

[(77, 79),
 (103, 85),
 (137, 85),
 (135, 109),
 (107, 109),
 (79, 113),
 (81, 135),
 (101, 131),
 (135, 123),
 (140, 139)]

In [21]:
longest_path_length # NOT off by 1

6318

In [22]:
# Check
total = 0
for i in range(len(longest_path) - 1):
    total += adj_dict[longest_path[i]][longest_path[i + 1]]
total

6318

Using a recursion is faster.

In [31]:
adj_dict_alt = {k: list(v.items()) for k, v in adj_dict.items()}

In [35]:
def do_dfs_recursive(adj_dict_alt, start, end, seen=set()):

    if start == end:
        return [0]
    
    seen.add(start)
    path_lengths = []

    for next_vertex, dist in adj_dict_alt[start]:
        if next_vertex not in seen:
            res = do_dfs_recursive(adj_dict_alt, next_vertex, end, seen)
            for pl in res:
                path_lengths.append(pl + dist)

    seen.remove(start)

    return path_lengths

In [38]:
path_lengths = do_dfs_recursive(adj_dict_alt, start, end)

In [39]:
max(path_lengths)

6318